In [1]:
from datetime import datetime
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from imutils import face_utils
from collections import Counter
import glob
import pandas as pd
import random
from datetime import datetime
import dlib

import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms

from Denoiser.GFPGANer import GFPGANer_Model
from retinaface import RetinaFace
from deepface import DeepFace
from deepface.commons import distance


%matplotlib inline

frame_x = 1250 #1250
frame_y = 750 #750

C:\Users\s.ezati\Anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
denoiser = GFPGANer_Model()

In [3]:
rgb_colors = []
font = cv2.FONT_HERSHEY_SIMPLEX
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [4]:
def make_square(img, min_size, fill_color=(200, 200, 200)):
    img_x_y =(img.shape[1]/img.shape[0])
    img = cv2.resize(np.array(img),(round(min_size*img_x_y),min_size))
    im = Image.fromarray(img)
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size -x) / 2), int((size -y) / 2)))
    return new_im

In [5]:
def Fix_Face(img,left_eye=None,right_eye=None):
    gray = img.copy()
    
    if left_eye is None and right_eye is None:
        try: m = mtcnn.detect_faces(gray)
        except ValueError: return img

        ex = m[0]['keypoints']['right_eye'][0]
        ey = m[0]['keypoints']['right_eye'][1]

        left_eye_center = (m[0]['keypoints']['left_eye'][0],m[0]['keypoints']['left_eye'][1])
        left_eye_x = m[0]['keypoints']['left_eye'][0]
        left_eye_y = m[0]['keypoints']['left_eye'][1]

        right_eye_center = (m[0]['keypoints']['right_eye'][0],m[0]['keypoints']['right_eye'][1])
        right_eye_x = m[0]['keypoints']['right_eye'][0]
        right_eye_y = m[0]['keypoints']['right_eye'][1]

        delta_x = right_eye_x - left_eye_x
        delta_y = right_eye_y - left_eye_y
        angle = np.arctan(delta_y/delta_x)
        angle = (angle * 180) / np.pi

        h, w = img.shape[:2]

        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, (angle), 1.0)

        rotated = cv2.warpAffine(img, M, (w, h))
        
    else:
        left_eye_x = left_eye[0]
        left_eye_y = left_eye[1]

        right_eye_x = right_eye[0]
        right_eye_y = right_eye[1]

        delta_x = right_eye_x - left_eye_x
        delta_y = right_eye_y - left_eye_y
        angle = np.arctan(delta_y/delta_x)
        angle = (angle * 180) / np.pi

        h, w = img.shape[:2]

        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, (angle), 1.0)

        rotated = cv2.warpAffine(img, M, (w, h))  
        
    return rotated

In [6]:
def Pre_Proc_Light(img):
    lightness = np.mean(img)
    if lightness < 95:
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    else: return img

In [7]:
dataset=datasets.ImageFolder('people/')
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} 
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] 
name_list = [] 
embedding_list = [] 

In [8]:
for imgs, idx in loader:
    
    faces = RetinaFace.detect_faces(np.array(imgs))

    for face in faces:

        save_flag = True
        try: score = faces[face]['score']*100
        except TypeError: break

        if score < 90: save_flag = False
        else: 
            x = faces[face]['facial_area'][0]
            y = faces[face]['facial_area'][1]
            w = faces[face]['facial_area'][2]
            h = faces[face]['facial_area'][3]

            img = np.array(imgs)[y:h, x:w].copy()
            w_resize = img.shape[1]
            h_resize = img.shape[0]
            try:
                x_right = round(abs(faces[face]['landmarks']['right_eye'][0]-x)*(300/w_resize))
                y_right = round(abs(faces[face]['landmarks']['right_eye'][1]-y)*(300/h_resize))

                x_left = round(abs(faces[face]['landmarks']['left_eye'][0]-x)*(300/w_resize))
                y_left = round(abs(faces[face]['landmarks']['left_eye'][1]-y)*(300/h_resize))
                        except ZeroDivisionError:
                z = 0
                       
            dist = np.linalg.norm((np.array([x_left,y_left])-np.array([x_right,y_right])),ord=2)
            
            
            img_x_y =(img.shape[1]/img.shape[0])
#             _, _, restored_img = denoiser.enhance(np.array(img))
            restored_img = Fix_Face(np.array(img),[x_left,y_left],[x_right,y_right])
            restored_img = cv2.resize(np.array(restored_img),(round(restored_img.shape[0]*img_x_y),restored_img.shape[0]))
            img = make_square(restored_img, 300)
           
    
        try: 
            embedding = DeepFace.represent(np.array(img),detector_backend=None,model_name='Facenet512')
            embedding_list.append(embedding) 
            name_list.append(idx_to_class[idx])
        except ValueError: continue

1/1 [==============================] - 0s 109ms/step


In [9]:
%matplotlib inline

imgs_original = np.array(cv2.imread('17.jpg'))
imgs_original = cv2.cvtColor(np.array(imgs_original),cv2.COLOR_BGR2RGB)
faces = RetinaFace.detect_faces(np.array(cv2.imread('17.jpg')))
imgs_list = []
for face in faces:

    save_flag = True
    try: score = faces[face]['score']*100
    except TypeError: break

    if score < 90: save_flag = False
    else: 
        x = faces[face]['facial_area'][0]
        y = faces[face]['facial_area'][1]
        w = faces[face]['facial_area'][2]
        h = faces[face]['facial_area'][3]
        
        imgs = imgs_original.copy()      
        img = np.array(imgs)[y:h, x:w].copy()
        w_resize = img.shape[1]
        h_resize = img.shape[0]
        img_x_y =(img.shape[1]/img.shape[0])

        img = cv2.resize(np.array(img),(round(300),300))
        x, y, restored_img = denoiser.enhance(np.array(img),only_center_face=True)
        restored_img = cv2.resize(np.array(restored_img),(round(restored_img.shape[0]*img_x_y),restored_img.shape[1]))
        restored_img = make_square(restored_img, 200)
        imgs_list.append(restored_img)


In [10]:
def preprocess(img):
    im = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)
    row, col = im.shape[:2]
    bottom = im[row-2:row, 0:col]
    mean = cv2.mean(bottom)[0]
    bordersize = 50

    top=bordersize
    bottom=bordersize
    left=bordersize
    right=bordersize

    border = cv2.copyMakeBorder(
        im,
        top=top,
        bottom=bottom,
        left=left,
        right=right,
        borderType=cv2.BORDER_CONSTANT,
        value=[255, 255, 255])

In [11]:
from threading import Thread

class ThreadedCamera(object):
    def __init__(self, source='cam:Sayda@0.0.0.0:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}', cv2.CAP_FFMPEG)

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

In [12]:
def Recognition(img,img_original,ROI_Region,thresh=1.04):
    now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
    img = np.array(Image.fromarray(img))
    img = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)

    faces = RetinaFace.detect_faces(img)

    for face in faces:

        save_flag = True
        try: score = faces[face]['score']*100
        except TypeError: break

        if score < 90: save_flag = False
        else: 
            x = faces[face]['facial_area'][0]
            y = faces[face]['facial_area'][1]
            w = faces[face]['facial_area'][2]
            h = faces[face]['facial_area'][3]
          
            resize_frame = img[y:h, x:w].copy()
            w_resize = resize_frame.shape[1]
            h_resize = resize_frame.shape[0]
            resize_frame = cv2.resize(resize_frame,(300,300))
            
            x_right = round(abs(faces[face]['landmarks']['right_eye'][0]-x)*(300/w_resize))
            y_right = round(abs(faces[face]['landmarks']['right_eye'][1]-y)*(300/h_resize))

            x_left = round(abs(faces[face]['landmarks']['left_eye'][0]-x)*(300/w_resize))
            y_left = round(abs(faces[face]['landmarks']['left_eye'][1]-y)*(300/h_resize))
            
            dist = np.linalg.norm((np.array([x_left,y_left])-np.array([x_right,y_right])),ord=2)
            
            if dist > 120 :
                
                img_x_y =(resize_frame.shape[1]/resize_frame.shape[0])
                _, _, restored_img = denoiser.enhance(np.array(resize_frame))
                restored_img = Fix_Face(np.array(restored_img),[x_left,y_left],[x_right,y_right])
                restored_img = cv2.resize(np.array(restored_img),(round(restored_img.shape[0]*img_x_y),restored_img.shape[0]))
                restored_img = make_square(restored_img, 300)
                
                try: embedding = DeepFace.represent(np.array(restored_img),detector_backend=None,model_name='Facenet512')
                except ValueError: continue

                person,person2,flag,candidates,candidates2,person_id = 'Unknown','Unknown',False,[],[],[]
                for indx, emb_db in enumerate(embedding_list):
                    res = np.linalg.norm((np.array(emb_db)-np.array(embedding)),ord=2)
 
                    if res < (thresh):
                        flag = True
                        person = name_list[indx]
                        candidates.append((res,person))
                        candidates2.append(person)
                    else: pass
                
                if len(candidates2) < 2: pass
                else: 
                    b = Counter(candidates2)
                    person2 = b.most_common(1)[0][0]

                if flag: 
                    color = (0,255,0)
                    person = min(candidates,key = lambda t: t[0])[1]
                    print("person")
                    print( person)
                    person_id.append(person)
                else: color = (0,0,255)   

                img_original = cv2.rectangle(np.array(img_original), (x+ROI_Region[0], y+ROI_Region[1],abs(w-(x)),abs(h-(y))), color, 2)
                img_original = cv2.putText(np.array(img_original),  f'{person}', (x+ROI_Region[0], y+ROI_Region[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2,cv2.LINE_AA)

    img_original = cv2.resize(img_original, (frame_x, frame_y))
    try: return img_original,img[y:h, x:w]
    except UnboundLocalError: return img_original,img_original

In [13]:
def mouse_click(event, x, y, flags, param):
    global ROI_Region
    global img_original2
    global img_save
    
    if event == cv2.EVENT_RBUTTONDOWN:
        ROI_Region = cv2.selectROI("Select Area",img_original2)
        print(ROI_Region)
        cv2.destroyAllWindows()
        print(ROI_Region)
    elif event == cv2.EVENT_MBUTTONDOWN:
        ROI_Region = [0,0,0,0]
    else: pass

In [14]:
# ROI_Region = [0,0,0,0]
# ROI_Region1 = [405, 150, 663, 549]
# ROI_Region2 = [266, 188, 639, 486]
ROI_Region1 = [0, 0, 0, 0]
ROI_Region2 = [0, 0,0, 0]
#(441, 114, 459, 487) : 121
#(396, 138, 483, 541) : 120
frame_list,counter = [],0

streamer_2 = ThreadedCamera('rtsp:Sayda@0.0.0.0')

In [ ]:
while True: 
    img_original2 = streamer_2.grab_frame()

    
    try: img_original2 = Image.fromarray(img_original2)
    except AttributeError: continue
       
    img_original2 = cv2.resize(np.array(img_original2), (frame_x, frame_y))

 
    img2 = img_original2.copy()
    img_save2 = img_original2.copy()
    
    
    
    try: 
        if ROI_Region2[0] == ROI_Region2[1] and ROI_Region2[2] == ROI_Region2[3]:
            if ROI_Region2[0] == 0 and ROI_Region2[0] == ROI_Region2[3]: pass
            else: 
                img2 = img2[int(ROI_Region2[1]):int(ROI_Region2[1]+ROI_Region2[3]),
                            int(ROI_Region2[0]):int(ROI_Region2[0]+ROI_Region2[2])]
        else: 
            img2 = img2[int(ROI_Region2[1]):int(ROI_Region2[1]+ROI_Region2[3]),
                        int(ROI_Region2[0]):int(ROI_Region2[0]+ROI_Region2[2])]
 
    except IndexError: pass
    
    
   
        
    if img_original2 is None: img_res2 = np.zeros([frame_y,frame_x,3],dtype=np.uint8)
    else: img_res2,_ = Recognition(img2,img_original2,ROI_Region2,23.56-3.5)
        

    img_res2 = cv2.resize(img_res2, (frame_x, frame_y))
    cv2.imshow('verification', np.array(img_res2))
    
    cv2.setMouseCallback('verification', mouse_click)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break

(67, 294, 104, 133)
(67, 294, 104, 133)
1/1 [==============================] - 0s 109ms/step
person
Shabnam
1/1 [==============================] - 0s 109ms/step
person
mr zamani
1/1 [==============================] - 0s 109ms/step
person
Mohammad
1/1 [==============================] - 0s 109ms/step


In [ ]:
# while True: 
#     img_original2 = streamer_2.grab_frame()

    
#     try: img_original2 = Image.fromarray(img_original2)
#     except AttributeError: continue
       
#     img_original2 = cv2.resize(np.array(img_original2), (frame_x, frame_y))

 
#     img2 = img_original2.copy()
#     img_save2 = img_original2.copy()
    
    
    
#     try: 
#         if ROI_Region[0] == ROI_Region[1] and ROI_Region[2] == ROI_Region[3]:
#             if ROI_Region[0] == 0 and ROI_Region[0] == ROI_Region[3]: pass
#             else: 
#                 img2 = img2[int(ROI_Region[1]):int(ROI_Region[1]+ROI_Region[3]),
#                             int(ROI_Region[0]):int(ROI_Region[0]+ROI_Region[2])]
#         else: 
#             img2 = img2[int(ROI_Region[1]):int(ROI_Region[1]+ROI_Region[3]),
#                         int(ROI_Region[0]):int(ROI_Region[0]+ROI_Region[2])]
 
#     except IndexError: pass
       
        
#     if img_original2 is None: img_res2 = np.zeros([frame_y,frame_x,3],dtype=np.uint8)
#     else: img_res2,_ = Recognition(img2,img_original2,ROI_Region,23.56-3.25)
        

#     img_res2 = cv2.resize(img_res2, (frame_x, frame_y))
#     cv2.imshow('verification', np.array(img_res2))
    
#     cv2.setMouseCallback('verification', mouse_click)
#     if cv2.waitKey(1) & 0xFF == ord('q'): 
#         cv2.destroyAllWindows()
#         streamer.end_frame()
#         break

In [ ]:
# while True: 
#     img_original1 = streamer_1.grab_frame()
#     img_original2 = streamer_2.grab_frame()
    
#     try: img_original1 = Image.fromarray(img_original1)
#     except AttributeError: pass
    
#     try: img_original2 = Image.fromarray(img_original2)
#     except AttributeError: continue
       
#     img_original1 = cv2.resize(np.array(img_original1), (frame_x, frame_y))
#     img_original2 = cv2.resize(np.array(img_original2), (frame_x, frame_y))

#     img1 = img_original1.copy()
#     img_save1 = img_original1.copy()
    
#     img2 = img_original2.copy()
#     img_save2 = img_original2.copy()
    
#     try: 
#         if ROI_Region1[0] == ROI_Region1[1] and ROI_Region1[2] == ROI_Region1[3]:
#             if ROI_Region1[0] == 0 and ROI_Region1[0] == ROI_Region1[3]: pass
#             else: 
#                 img1 = img1[int(ROI_Region1[1]):int(ROI_Region1[1]+ROI_Region1[3]),
#                             int(ROI_Region1[0]):int(ROI_Region1[0]+ROI_Region1[2])]
#         else: 
#             img1 = img1[int(ROI_Region1[1]):int(ROI_Region1[1]+ROI_Region1[3]),
#                         int(ROI_Region1[0]):int(ROI_Region1[0]+ROI_Region1[2])]
 
#     except IndexError: pass
    
#     try: 
#         if ROI_Region2[0] == ROI_Region2[1] and ROI_Region2[2] == ROI_Region2[3]:
#             if ROI_Region2[0] == 0 and ROI_Region2[0] == ROI_Region2[3]: pass
#             else: 
#                 img2 = img2[int(ROI_Region2[1]):int(ROI_Region2[1]+ROI_Region2[3]),
#                             int(ROI_Region2[0]):int(ROI_Region2[0]+ROI_Region2[2])]
#         else: 
#             img2 = img2[int(ROI_Region2[1]):int(ROI_Region2[1]+ROI_Region2[3]),
#                         int(ROI_Region2[0]):int(ROI_Region2[0]+ROI_Region2[2])]
 
#     except IndexError: pass
    
    
#     if img_original1 is None: img_res1 = np.zeros([frame_y,1250,3],dtype=np.uint8)
#     else: img_res1,_ = Recognition(img1,img_original1,ROI_Region1,23.56-4.5)
        
#     if img_original2 is None: img_res2 = np.zeros([frame_y,frame_x,3],dtype=np.uint8)
#     else: img_res2,_ = Recognition(img2,img_original2,ROI_Region2,23.56-4.7)
        

#     img_res1 = cv2.resize(img_res1, (frame_x, frame_y))
#     img_res2 = cv2.resize(img_res2, (frame_x, frame_y))
#     img = np.concatenate((img_res1, img_res2), axis=1)
#     img = cv2.resize(img, (frame_x, frame_y))
#     cv2.imshow('verification', np.array(img))
    
#     cv2.setMouseCallback('verification', mouse_click)
#     if cv2.waitKey(1) & 0xFF == ord('q'): 
#         cv2.destroyAllWindows()
#         streamer.end_frame()
#         break